In [1]:
import pandas as pd

In [2]:
from nsepy import get_history as gh
import datetime as dt

In [3]:
import yfinance as yf

In [ ]:
import datetime as dt

start = dt.datetime(2021, 7, 1)
end = dt.datetime(2022, 1, 27)
symbol = 'AXISBANK.NS'  # Stock symbol in uppercase

stk_data = yf.download(symbol, start=start, end=end)

In [ ]:
stk_data=stk_data[["Open","High","Low","Close"]]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

In [ ]:
len(data1)

In [ ]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])

In [ ]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [ ]:
def combination(dataset,listt):
    print(listt)
    datasetTwo=dataset[listt]
    test_obs=28
    train=datasetTwo[:-test_obs]
    test=datasetTwo[-test_obs:]
    from statsmodels.tsa.api import VAR
    for i in [1,2,3,4,5,6,7,8,9,10]:
        model=VAR(train)
        results=model.fit(i)
        print('order=',i)
        print('AIC:',results.aic)
        print('BIC:',results.bic)
        print()
    x=model.select_order(maxlags=12)
    order=x.selected_orders["aic"]
    result=model.fit(order)
    #result.summary()
    lagged_Values=train.values[-order:]
    pred=result.forecast(y=lagged_Values,steps=28)
    preds=pd.DataFrame(pred,columns=listt)
    preds.to_csv("varforecasted_{}.csv".format(test_obs))
    from sklearn.metrics import mean_squared_error
    rmse=round(mean_squared_error(test,pred,squared=False))
    from sklearn.metrics import mean_absolute_percentage_error
    mape=mean_squared_error(test,pred)
    performance["Model"].append(listt)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(order)
    performance["Test"].append(test_obs)
    perf=pd.DataFrame(performance)
    return perf,result,pred

In [ ]:
listt=["Open","High","Low","Close"]

In [ ]:
perf,result,pred=combination(data1,listt)


In [ ]:
data1

In [ ]:
perf